# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [2]:
import numpy as np

In [152]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, self.d)
                
    """Task 1.1"""
    """ Called to implement the unary arithmetic operations (-, +, abs() and ~). """
    def __neg__(self) -> "Dual":  # unary arithmetic -
        return -1 * self

    def __pos__(self) -> "Dual":  # unary arithmetic +
        return self

    def __abs__(self) -> "Dual":  # abs()
        if self.value != 0:
            return Dual(abs(self.value), self.d * np.sign(self.value))
        else:  # case value = 0
            return Dual(0, 0)

    #TODO: write invert method
    """Something strange"""
    def __invert__(self) -> "Dual":  # unary arithmetic ~
        return Dual(~self.value, -1 * self.d)  # Dual(-1*(self.value+1), -1*(self.d))

    def __truediv__(self, other: Union["Dual", Number]):  # div by zero! 
        match other:
            case Dual(o_value, o_d):
                if o_value == 0:
                    raise ZeroDivisionError
                else:
                    return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value * o_value))
            case Number():
                if other == 0:
                    raise ZeroDivisionError
                else:
                    return Dual(self.value / float(other), self.d / float(other))
           

    def __pow__(self, k: float) -> "Dual":  # for numbers only
        if self.value != 0:
            return Dual(self.value**k, self.d * k * self.value**(k - 1))
        else:
            raise ValueError
    
    # ???
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, self.value ** o_value * (other * self.log()).d)
            case Number():
                    return Dual(float(other) ** self.value, float(other)**self.value * np.log(float(other)))
                
    # ???
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        try:
            match other:
                case Dual(o_value, o_d):
                    return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / (o_value * o_value))
                case Number():
                    return Dual(float(other) / self.value, -self.d * float(other) / (self.value * self.value))
        except ZeroDivisionError:
            print("Деление на ноль")
            exit(-1)

    # ???
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, -self.d)
    ##################################################     

    """Task 1.2"""
    def exp(self) -> "Dual":
        return Dual(np.exp(self.value), self.d * np.exp(self.value))

    def cos(self) -> "Dual":
        return Dual(np.cos(self.value), -self.d * np.sin(self.value))

    def sin(self) -> "Dual":
        return Dual(np.sin(self.value), self.d * np.cos(self.value))

    def log(self) -> "Dual":
        if self.value > 0:
            return Dual(np.log(self.value), self.d / self.value)
        else:
            raise ValueError


def diff_1(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 
    
"""Task 1.5"""
# differentiation with 2 variables
def diff_2(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x, y: [func(Dual(x, 1.0), Dual(y, 0.0)).d, func(Dual(x, 0.0), Dual(y, 1.0)).d]

# differentiation with 3 variables
def diff_3(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x, y, z: [func(Dual(x, 1.0), Dual(y, 0.0), Dual(z, 0.0)).d, func(Dual(x, 0.0), Dual(y, 1.0), Dual(z, 0.0)).d, func(Dual(x, 0.0), Dual(y, 0.0), Dual(z, 1.0))]

# diff with several variables
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    def newfunc(*args):
        answers = []
        for j in range(0, len(args)):
            ans = func(*[Dual(args[i], float(i==j)) for i in range(0, len(args))])
            if type(ans) != Dual:
                ans = 0
            else:
                ans = ans.d
            answers.append(ans)
        return answers
    return newfunc

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [144]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff_1(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [27]:
import unittest
import pytest

In [41]:
# unary arithmetic -
assert -Dual(0, 1) == Dual(0, -1)
assert -Dual(1, 0) == Dual(-1, 0)
assert -Dual(2, -5) == Dual(-2, 5)
assert -Dual(-2, 5) == Dual(2, -5)

In [42]:
# unary arithmetic +
assert +Dual(0, 1) == Dual(0, 1)
assert +Dual(-3, -1) == Dual(-3, -1)

In [44]:
# abs()
assert abs(Dual(2, 1)) == Dual(2, 1)
assert abs(Dual(-2, 1)) == Dual(2, -1)
assert abs(Dual(-2, -2)) == Dual(2, 2)
assert abs(Dual(0, 1)) == Dual(0, 0)
assert abs(Dual(0, 0)) == Dual(0, 0)

In [68]:
# unary arithmetic ~
assert ~Dual(2, 1) == Dual(-3, -1)
assert ~Dual(0, 1) == Dual(-1, -1)
assert ~Dual(1, 1) == Dual(-2, -1)
assert ~Dual(2, 2) == Dual(-3, -2)
assert ~Dual(3, 3) == Dual(-4, -3)

assert ~Dual(0, 0) == Dual(-1, 0)
assert ~Dual(1, 0) == Dual(-2, 0)
#unit test for invert method for Dual with negative value
assert ~Dual(-2, 1) == Dual(1, -1)
assert ~Dual(-1, 1) == Dual(0, -1)
assert ~Dual(-2, 2) == Dual(1, -2)
assert ~Dual(-3, 3) == Dual(2, -3)

In [79]:
Dual(2, 1)/ Dual(2, 1)

Dual(value=1.0, d=0.0)

In [92]:
Dual(-2, 3) / Dual(2, 2)

Dual(value=-1.0, d=2.5)

In [100]:
# dev
with pytest.raises(ZeroDivisionError):
    Dual(5, 1) / 0
with pytest.raises(ZeroDivisionError):
    Dual(5, 1) / Dual(0,1)
with pytest.raises(ZeroDivisionError):
    Dual(0, 2) / Dual(0,1)
    
assert Dual(2, 1) / Dual(2, 1) == Dual(1, 0)
assert Dual(2, 1) / Dual(2, 2) == Dual(1, -0.5)
assert Dual(2, 2) / Dual(4, 1) == Dual(0.5, 0.375)

assert Dual(2, -3) / Dual(2, 1) == Dual(1, -2)
assert Dual(-2, 3) / Dual(2, 2) == Dual(-1, 2.5)
assert Dual(2, 5) / Dual(-5, 2) == Dual(-0.4, -29/25)
assert Dual(2, 4) / Dual(2, -1) == Dual(1, 2.5)
assert Dual(-2, -4) / -2 == Dual(1, 2)

In [34]:
# pow
assert Dual(2, 1)**2 == Dual(4, 4)
assert Dual(2, 1)**3 == Dual(8, 12)
assert Dual(2, 1)**4 == Dual(16, 32)
assert Dual(2, 1)**5 == Dual(32, 80)

assert Dual(2, 1)** (-1) == Dual(0.5, -0.25)
assert Dual(2, 1) ** 0 == Dual(1, 0)
with pytest.raises(ValueError):
    Dual(0, 1) ** 2

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [39]:
# exp()
assert Dual(2, 1).exp() == Dual(np.exp(2), np.exp(2))
assert Dual(2, -2).exp() == Dual(np.exp(2), -2 * np.exp(2))
assert Dual(2, 0).exp() == Dual(np.exp(2), 0)
assert Dual(-4, 0.5).exp() == Dual(np.exp(-4), 0.5 * np.exp(-4))
assert Dual(0, 1).exp() == Dual(np.exp(0), 1 * np.exp(0))

In [109]:
# cos() ?
#unit-test for cos method for Dual
assert Dual(2, 1).cos() == Dual(np.cos(2), -np.sin(2))
assert Dual(2, 2).cos() == Dual(np.cos(2), -2 * np.sin(2))
assert Dual(-2, -3).cos() == Dual(np.cos(-2), -(-3) * np.sin(-2))
assert Dual(-2, 4).cos() == Dual(np.cos(-2), -4 * np.sin(-2))
assert Dual(2, -5).cos() == Dual(np.cos(2), - (-5) * np.sin(2))
assert Dual(0, 1).cos() == Dual(np.cos(0), -np.sin(0))
assert Dual(2, 0).cos() == Dual(np.cos(2), 0)
assert Dual(0, 0).cos() == Dual(np.cos(0), 0)

In [108]:
# sin()
#unit-test for sin method for Dual
assert Dual(2, 1).sin() == Dual(np.sin(2), np.cos(2))
assert Dual(10, 4).sin() == Dual(np.sin(10), 4 * np.cos(10))
assert Dual(2, 5).sin() == Dual(np.sin(2), 5 * np.cos(2))

assert Dual(0, 1).sin() == Dual(0, 1)

assert Dual(-2, 1).sin() == Dual(np.sin(-2), np.cos(-2))
assert Dual(2, -2).sin() == Dual(np.sin(2), -2 * np.cos(2))
assert Dual(-2, 0).sin() == Dual(np.sin(-2), 0)
assert Dual(0, 0).sin() == Dual(np.sin(0), 0)

In [117]:
Dual(10, 3).log()

Dual(value=2.302585092994046, d=0.3)

In [118]:
# log()
assert Dual(2, 1).log() == Dual(np.log(2), 1/2)
assert Dual(2, 2).log() == Dual(np.log(2), 2/2)
assert Dual(10, 3).log() == Dual(np.log(10), 3/10)
assert Dual(4, -3).log() == Dual(np.log(4), -3/4)
with pytest.raises(ValueError):
    Dual(0, 1).log()
with pytest.raises(ValueError):
    Dual(-2, 1).log()

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [11]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [147]:
# ваш код
def f1(x: float) -> float:
    return 5 * x * x * x - (5 * x - 3) * x**(-2) + 7 * x + 1

f_diff1 = diff_1(f1)
# значение производной в точке x = 2
print(derivative(f1, 2.), f_diff1(2))


72.33333333333333 64.0


In [148]:
def f(x: float) -> float:
    return 5 * x * np.log(x)

f_diff = diff_1(f)
print(derivative(f, 2.), f_diff(2.))


8.239592165010823 8.465735902799727


In [154]:
def f(x: float) -> float:
    return 3 * x * x + 0.4 * x + 2 + np.cos(x)

f_diff = diff_1(f)
print(derivative(f, 4.), f_diff(4.))


25.036827341031838 25.156802495307925


In [156]:
def f(x: float) -> float:
    return np.cos(x) + np.sin(x)

f_diff = diff_1(f)
print(derivative(f, 3.4), f_diff(3.4))

-0.5985022044389204 -0.7112570905526299


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [ ]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [151]:
# ваш код
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]

[10.0, 5.0, 1.0]